<a href="https://colab.research.google.com/github/warlords94/personal-coding/blob/main/CCT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q tensorflow-addons

     |████████████████████████████████| 1.1 MB 10.6 MB/s 


In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np

In [ ]:
positional_emb = True
conv_layers = 2
projection_dim = 128

num_heads = 2
transformer_units = [
    projection_dim,
    projection_dim,
]
transformer_layers = 2
stochastic_depth_rate = 0.1

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128
num_epochs = 30
image_size = 32

In [ ]:
num_classes = 10
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

170508288/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 10)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 10)


In [ ]:
class CCTTokenizer(layers.Layer):
    def __init__(
        self,
        kernel_size=3,
        stride=1,
        padding=1,
        pooling_kernel_size=3,
        pooling_stride=2,
        num_conv_layers=conv_layers,
        num_output_channels=[64, 128],
        positional_emb=positional_emb,
        **kwargs,
    ):
        super(CCTTokenizer, self).__init__(**kwargs)

        # This is our tokenizer.
        self.conv_model = keras.Sequential()
        for i in range(num_conv_layers):
            self.conv_model.add(
                layers.Conv2D(
                    num_output_channels[i],
                    kernel_size,
                    stride,
                    padding="valid",
                    use_bias=False,
                    activation="relu",
                    kernel_initializer="he_normal",
                )
            )
            self.conv_model.add(layers.ZeroPadding2D(padding))
            self.conv_model.add(
                layers.MaxPool2D(pooling_kernel_size, pooling_stride, "same")
            )

        self.positional_emb = positional_emb

    def call(self, images):
        outputs = self.conv_model(images)
        # After passing the images through our mini-network the spatial dimensions
        # are flattened to form sequences.
        reshaped = tf.reshape(
            outputs,
            (-1, tf.shape(outputs)[1] * tf.shape(outputs)[2], tf.shape(outputs)[-1]),
        )
        return reshaped

    def positional_embedding(self, image_size):
        # Positional embeddings are optional in CCT. Here, we calculate
        # the number of sequences and initialize an `Embedding` layer to
        # compute the positional embeddings later.
        if self.positional_emb:
            dummy_inputs = tf.ones((1, image_size, image_size, 3))
            dummy_outputs = self.call(dummy_inputs)
            sequence_length = tf.shape(dummy_outputs)[1]
            projection_dim = tf.shape(dummy_outputs)[-1]

            embed_layer = layers.Embedding(
                input_dim=sequence_length, output_dim=projection_dim
            )
            return embed_layer, sequence_length
        else:
            return None

In [ ]:
# Referred from: github.com:rwightman/pytorch-image-models.
class StochasticDepth(layers.Layer):
    def __init__(self, drop_prop, **kwargs):
        super(StochasticDepth, self).__init__(**kwargs)
        self.drop_prob = drop_prop

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_prob
            shape = (tf.shape(x)[0],) + (1,) * (tf.shape(x).shape[0] - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
# Note the rescaling layer. These layers have pre-defined inference behavior.
data_augmentation = keras.Sequential(
    [
        layers.Rescaling(scale=1.0 / 255),
        layers.RandomCrop(image_size, image_size),
        layers.RandomFlip("horizontal"),
    ],
    name="data_augmentation",
)

In [ ]:
def create_cct_model(
    image_size = image_size,
    input_shape = input_shape,
    num_heads=num_heads,
    projection_dim =projection_dim,
    transformer_units = transformer_units,):
  inputs = layers.Input(input_shape)
  augmented = data_augmentation(inputs)
  cct_tokenizer = CCTTokenizer()
  encoded_patches = cct_tokenizer(augmented)
  if positional_embed:
    pos_embed,seq_length = cct_tokenizer.positional_embedding(image_size)
    positions = tf.range(start=0,limit=seq_length, delta=1)
    positional_embeddings = pos_embed(positions)
    encoded_patches += positional_embeddings
  dpr = [x for x in np.linspace(0,stochastic_depth_rate,transformer_layers)]
  for i in range(transformer_layers):
    x1 = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)
    attention_output = layers.MultiHeadAttention(num_heads = num_heads , key_dim = projection_dim, dropout =0.1)(x1,x1)
    attention_output = StochasticDepth(dpr[i])(attention_output)
    x2 = layers.Add()([attention_output,encoded_patches])
    x3 = layers.LayerNormalization(epsilon=1e-5)(x2)
    x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
    x3 = StochasticDepth(dpr[i])(x3)
    encoded_patches = layers.Add()([x3, x2])
  representation = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)
  attention_weights = tf.nn.softmax(layers.Dense(1)(representation),axis =1)
  weighted_representation = tf.matmul( attention_weights, representation, transpose_a=True)
  weighted_representation = tf.squeeze(weighted_representation,-2)
  logits = layers.Dense(num_classes)(weighted_representation)
  model = keras.Model(inputs = inputs, outputs =logits)
  return model





In [ ]:
def run_experiment(model):
  optimizer = tfa.optimizers.AdamW(learning_rate =0.001,weight_decay=0.0001)
  model.compile(
      optimizer=optimizer,
      loss=keras.losses.CategoricalCrossentropy(
          from_logits = True, label_smoothing =0.1
      ),
      metrics =[
          keras.metrics.CategoricalAccuracy(name ='accuracy'),
          keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),


      ],
  )
  history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        
    )
  _,accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
  print(f"Test accuracy: {round(accuracy * 100, 2)}%")
  print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

  return history
  


In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.CategoricalCrossentropy(
            from_logits=True, label_smoothing=0.1
        ),
        metrics=[
            keras.metrics.CategoricalAccuracy(name="accuracy"),
            keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history


cct_model = create_cct_model()
history = run_experiment(cct_model)

In [ ]:
cct_model = create_cct_model()
history = run_experiment(cct_model)

Epoch 1/30
352/352 [==============================] - 396s 1s/step - loss: 1.9354 - accuracy: 0.3209 - top-5-accuracy: 0.8250 - val_loss: 1.7280 - val_accuracy: 0.4176 - val_top-5-accuracy: 0.9092
Epoch 2/30
352/352 [==============================] - 388s 1s/step - loss: 1.6261 - accuracy: 0.4744 - top-5-accuracy: 0.9225 - val_loss: 1.5542 - val_accuracy: 0.4982 - val_top-5-accuracy: 0.9348
Epoch 3/30
352/352 [==============================] - 392s 1s/step - loss: 1.4920 - accuracy: 0.5378 - top-5-accuracy: 0.9439 - val_loss: 1.4180 - val_accuracy: 0.5778 - val_top-5-accuracy: 0.9570
Epoch 4/30
352/352 [==============================] - 390s 1s/step - loss: 1.3917 - accuracy: 0.5906 - top-5-accuracy: 0.9568 - val_loss: 1.4053 - val_accuracy: 0.5792 - val_top-5-accuracy: 0.9646
Epoch 5/30
352/352 [==============================] - 387s 1s/step - loss: 1.3342 - accuracy: 0.6190 - top-5-accuracy: 0.9622 - val_loss: 1.2562 - val_accuracy: 0.6570 - val_top-5-accuracy: 0.9722
Epoch 6/30
352/